In [1]:
import plotly
import plotly.graph_objects as go
import numpy as np
import scipy
import scipy.optimize
import scipy.stats
import ipywidgets as widgets

In [2]:
def parse_file(file: str):
    with open(file) as data:
        return [line.rstrip("\n").split(" ") for line in data]

In [3]:
parsed = parse_file("data") + parse_file("data2") + parse_file("data3") + parse_file("fast") + parse_file("medium")

In [4]:
swipe = np.array([line[1:] for line in parsed if line[0] == "Cancel"], dtype=np.float)
nonswipe = np.array([line[1:] for line in parsed if line[0] == "End"], dtype=np.float)

In [5]:
x = swipe[:, 0]
y = swipe[:, 1]
z = swipe[:, 2]

In [6]:
go.FigureWidget([
    go.Scatter3d(x=swipe[:, 0], y=swipe[:, 1], z=swipe[:, 2], mode="markers", marker={"size": 2}),
    go.Scatter3d(x=nonswipe[:, 0], y=nonswipe[:, 1], z=nonswipe[:, 2], mode="markers", marker={"size": 2}),
])

FigureWidget({
    'data': [{'marker': {'size': 2},
              'mode': 'markers',
              'type': 'sc…

In [7]:
z.max()

0.4999557083356194

In [8]:
fig = go.FigureWidget([
    go.Scatter(x=swipe[:, 0], y=swipe[:, 1], mode="markers", marker=dict(size=3)),
    go.Scatter(x=nonswipe[:, 0], y=nonswipe[:, 0], mode="markers", marker=dict(size=3))
])

In [9]:
def adjust_z_range(z):
    step = 0.016 * 3
    
    with fig.batch_update():
        swipe_in_range = (swipe[:, 2] > z) & (swipe[:, 2] <= z + step)
        filtered_swipe = swipe[swipe_in_range, :]

        swipe_scatter = fig.data[0]
        swipe_scatter.x = filtered_swipe[:, 0]
        swipe_scatter.y = filtered_swipe[:, 1]

        nonswipe_in_range = (nonswipe[:, 2] > z) & (nonswipe[:, 2] <= z + step)
        filtered_nonswipe = nonswipe[nonswipe_in_range, :]

        nonswipe_scatter = fig.data[1]
        nonswipe_scatter.x = filtered_nonswipe[:, 0]
        nonswipe_scatter.y = filtered_nonswipe[:, 1]
        
        fig.layout.xaxis.range = (20, 90)
        fig.layout.yaxis.range = (-70, 70)

In [10]:
widgets.interact(adjust_z_range, z=(0.0, 0.7, 0.016))

interactive(children=(FloatSlider(value=0.336, description='z', max=0.7, step=0.016), Output()), _dom_classes=…

<function __main__.adjust_z_range(z)>

In [11]:
fig

FigureWidget({
    'data': [{'marker': {'size': 3},
              'mode': 'markers',
              'type': 'sc…

In [12]:
go.FigureWidget([
    go.Scatter(x=swipe[:, 0], y=swipe[:, 2], mode="markers", marker=dict(size=3))
])

FigureWidget({
    'data': [{'marker': {'size': 3},
              'mode': 'markers',
              'type': 'sc…

### There is a clear straight line edge on the left side
### The line is from about (x=50, z=0) to (x=27, z=0.5), the function is x = -(50 - 27)/0.5 * z + 50

In [13]:
def x_from_zy(parameters):
    return -(parameters[0] - parameters[1]) / 0.5 * z + parameters[0] + parameters[2] * y

In [14]:
def unequal_loss(predict, truth):
    return np.where(truth > predict, 0.01 * (truth - predict), predict - truth).mean()

In [15]:
def optimize_target_x(p):
    return unequal_loss(x_from_zy(p), x)

In [16]:
result = scipy.optimize.minimize(optimize_target_x, (50, 27, 0), method='nelder-mead')
result

 final_simplex: (array([[5.04205840e+01, 2.63865212e+01, 9.75476379e-05],
       [5.04205495e+01, 2.63866091e+01, 9.76971137e-05],
       [5.04205961e+01, 2.63865560e+01, 9.77068289e-05],
       [5.04206069e+01, 2.63864540e+01, 9.75894081e-05]]), array([0.05958784, 0.05958785, 0.05958785, 0.05958785]))
           fun: 0.05958784238916305
       message: 'Optimization terminated successfully.'
          nfev: 103
           nit: 54
        status: 0
       success: True
             x: array([5.04205840e+01, 2.63865212e+01, 9.75476379e-05])

In [17]:
go.FigureWidget([
    go.Scatter3d(x=x, y=y, z=z, mode="markers", marker={"size": 3}),
    go.Scatter3d(x=x_from_zy(result.x), y=y, z=z, mode="markers", marker={"size": 2})
])

FigureWidget({
    'data': [{'marker': {'size': 3},
              'mode': 'markers',
              'type': 'sc…

In [18]:
go.FigureWidget([
    go.Scatter(x=x, y=z, mode="markers", marker={"size": 3}),
    go.Scatter(x=x_from_zy(result.x), y=z, mode="markers", marker={"size": 2})
])

FigureWidget({
    'data': [{'marker': {'size': 3},
              'mode': 'markers',
              'type': 'sc…

In [19]:
def parameter_from_xz(x, z):
    """Suppose 50 is accurate, calculate the parameter from x and z"""
    return 50 + 0.5 * (x - 50) / z

In [20]:
parameter_from_xz(40.66667, 0.2)

26.66667500000001

In [21]:
parameter_from_xz(45.3333, 0.099985)

26.662999449917496

In [22]:
go.FigureWidget([
    go.Scatter(x=swipe[:, 1], y=swipe[:, 2], mode="markers", marker=dict(size=3))
])

FigureWidget({
    'data': [{'marker': {'size': 3},
              'mode': 'markers',
              'type': 'sc…

### There is a clear straight line edge one the left
### The line is from about (y=-50, z=0) to (y=-25, z=0.5), the function is y = (50 - 25)/0.5 * z - 50

In [23]:
def y_from_zx(parameters):
    return (parameters[0] - parameters[1]) / 0.5 * z - parameters[0] + parameters[2] * x

In [24]:
def optimize_target_y(p):
    return unequal_loss(y_from_zx(p), y)

In [25]:
result_y = scipy.optimize.minimize(optimize_target_y, [50.0, 25.0, 0.0], method='BFGS')
result_y

      fun: 0.21600012552215317
 hess_inv: array([[ 14.39274077, -25.73016012,  -0.91722463],
       [-25.73016012,  49.8423685 ,   1.77691105],
       [ -0.91722463,   1.77691105,   0.06344738]])
      jac: array([0.00056843, 0.00027058, 0.01378711])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 722
      nit: 10
     njev: 142
   status: 2
  success: False
        x: array([4.99206902e+01, 2.52391427e+01, 1.90620662e-03])

In [26]:
go.FigureWidget([
    go.Scatter3d(x=x, y=y, z=z, mode="markers", marker={"size": 2}),
    go.Scatter3d(x=x, y=y_from_zx(result_y.x), z=z, mode="markers", marker={"size": 2})
])

FigureWidget({
    'data': [{'marker': {'size': 2},
              'mode': 'markers',
              'type': 'sc…

In [27]:
go.FigureWidget([
    go.Scatter(x=y, y=z, mode="markers", marker={"size": 3}),
    go.Scatter(x=y_from_zx(result_y.x), y=z, mode="markers", marker={"size": 2})
])

FigureWidget({
    'data': [{'marker': {'size': 3},
              'mode': 'markers',
              'type': 'sc…

In [28]:
def parameter_from_yz(y, z):
    """Suppose 50 is accurate, calculate the parameter from y and z"""
    return 50 - 0.5 * (y + 50) / z

In [29]:
parameter_from_yz(-45, 0.1001303)

25.03253261000916

In [30]:
parameter_from_yz(-32, 0.3663615)

25.43409992589287

In [31]:
parameter_from_yz(-30.33333, 0.3997435)

25.400888319634966

In [32]:
def edge(t):
    a = 25
    return (a - 50) / 0.5 * t + 50

In [33]:
def is_swipe(x, y, t):
    edge_x = edge(t)
    edge_y_min = -edge(t)
    edge_y_max = edge(t)
    
    return (x > edge_x) & (edge_y_min < y) & (y < edge_y_max) & (t < 0.5)

In [34]:
(~is_swipe(x, y, z)).nonzero()

(array([105, 168, 180, 249, 718, 740, 741, 752, 799]),)

In [35]:
swipe[741]

array([ 33.        , -30.        ,   0.40024608])

In [36]:
edge(0.40024608)

29.987696

**Only a few exceptions and they very near the edge**

In [37]:
is_swipe(nonswipe[:, 0], nonswipe[:, 1], nonswipe[:, 2]).nonzero()

(array([  2,   8,   9,  92,  98, 105, 132, 244, 245, 267, 280, 282, 291,
        293, 295, 301, 309, 311, 319, 326, 346, 353, 401, 448, 449, 451,
        458, 486, 488, 490, 492, 493, 495, 496, 499, 500, 501, 503, 504,
        509, 510, 511, 519, 520, 521, 523, 526, 534, 535, 536, 537, 538,
        539, 540, 542, 543, 544, 545, 548, 551, 553, 555, 556, 558, 559,
        588, 593, 608, 609, 614, 617]),)

**Exceptions are more**

## Simulation

Now we try to simulation the swipe action with different angle, distance and release time, to see if we can get similar graph from the experiment.

In [124]:
def simulate(n):
    max_angle = 60.0 / 180.0 * np.pi
    angle = np.random.rand(n, 1) * 2 * max_angle - max_angle
    max_distance = np.random.rand(n, 1) * 100
    release_time = np.random.rand(n, 1) * 0.7 + 0.001
        
    delta = 1 / 60
    t = np.arange(0, 0.8, delta)[None, :]
    
    move_distance = np.where(t < release_time, max_distance / release_time * t, np.full_like(max_distance, np.nan))
    
    x = move_distance * np.cos(angle)
    y = move_distance * np.sin(angle)
    
    return x, y, t

In [105]:
def group_simulation(x, y, t):
    n = x.shape[0]
    t_count = t.shape[1]
        
    is_swipe_array = np.where(np.isnan(x), np.full_like(x, False), is_swipe(x, y, t))
    is_swipe_per_touch = is_swipe_array.any(axis=1)
    
    swipe_xs = []
    swipe_ys = []
    swipe_ts = []
    nonswipe_xs = []
    nonswipe_ys = []
    nonswipe_ts = []
    for i in range(n):
        if is_swipe_per_touch[i]:
            # Find the first time that is recognized as swipe
            for j in range(t_count):
                if is_swipe_array[i, j]:
                    swipe_xs.append(x[i, j])
                    swipe_ys.append(y[i, j])
                    swipe_ts.append(t[0, j])
                    break
        else:
            # Use the release time
            for j in range(t_count):
                if np.isnan(x[i, j]):
                    k = j - 1  # The minimum release time is 0.001, this makes sure j = 0 is never an NaN
                    nonswipe_xs.append(x[i, k])
                    nonswipe_ys.append(y[i, k])
                    nonswipe_ts.append(t[0, k])
                    break
                    
    return np.array([swipe_xs, swipe_ys, swipe_ts]).T, np.array([nonswipe_xs, nonswipe_ys, nonswipe_ts]).T

In [125]:
simulate_swipe, simulate_nonswipe = group_simulation(*simulate(20000))

/Users/panyusheng/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning:

invalid value encountered in greater

/Users/panyusheng/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:6: RuntimeWarning:

invalid value encountered in less



In [126]:
go.Figure([
    go.Scatter3d(x=simulate_swipe[:, 0], y=simulate_swipe[:, 1], z=simulate_swipe[:, 2], mode="markers", marker=dict(size=2)),
    go.Scatter3d(x=simulate_nonswipe[:, 0], y=simulate_nonswipe[:, 1], z=simulate_nonswipe[:, 2], mode="markers", marker=dict(size=2))
])